In [9]:
import math

In [1]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [3]:
import pandas as pd
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)
print(train_data)

    年龄 有工作 有自己的房子 信贷情况 类别
0   青年   否      否   一般  否
1   青年   否      否    好  否
2   青年   是      否    好  是
3   青年   是      是   一般  是
4   青年   否      否   一般  否
5   中年   否      否   一般  否
6   中年   否      否    好  否
7   中年   是      是    好  是
8   中年   否      是  非常好  是
9   中年   否      是  非常好  是
10  老年   否      是  非常好  是
11  老年   否      是    好  是
12  老年   是      否    好  是
13  老年   是      否  非常好  是
14  老年   否      否   一般  否


In [117]:
def CalEnt(data):
    total = len(data)
    cate_sets = set(data['类别'])
    ent = 0
    for cate in cate_sets:
        sub_data = data[data['类别']==cate]
        ent -= len(sub_data)/total*math.log(len(sub_data)/total,2)
    return ent

def InfoGain(data,feature_name):
    total = len(data)
    total_ent = CalEnt(data)
    feature_sets = set(data[feature_name])
    ent = 0
    for feature in feature_sets:
        sub_data = data[data[feature_name]==feature]
        ent += len(sub_data)/total*CalEnt(sub_data)
    return total_ent - ent

def InfoGainRatio(data,feature_name):
    total = len(data)
    total_ent = CalEnt(data)
    feature_sets = set(data[feature_name])
    ent = 0
    den = 0
    for feature in feature_sets:
        sub_data = data[data[feature_name]==feature]
        ent += len(sub_data)/total*CalEnt(sub_data)
        den -= len(sub_data)/total*math.log(len(sub_data)/total,2)
    return (total_ent - ent)/den


print(InfoGain(train_data,"年龄"))
print(InfoGain(train_data,"有工作"))
print(InfoGain(train_data,"有自己的房子"))
print(InfoGain(train_data,"信贷情况"))

0.08300749985576883
0.32365019815155627
0.4199730940219749
0.36298956253708536


In [119]:
class TreeNode(object):
    def __init__(self,split_feature=None,children=None,cate=None):
        self.split_feature = split_feature
        self.children = dict()
        self.cate = cate
        
class DesitionTree(object):
    def __init__(self,train,labels,ep,criterion=InfoGainRatio):
        self.train = train
        self.root = TreeNode()
        self.labels = labels
        self.ep = ep
        self.criterion = criterion

    def BuildTree(self,data):
        if len(data) == 0:
            return None
        if set(data['类别'])==1:
            return TreeNode(cate = data['类别'].mode()[0])
        info_gain = []
        for feature_name in self.labels[:-1]:
            info_gain.append(self.criterion(data,feature_name))
        if max(info_gain)<self.ep:
            return TreeNode(cate=data['类别'].mode()[0])
        split_feature = self.labels[info_gain.index(max(info_gain))]
        node = TreeNode(split_feature = split_feature,cate=data['类别'].mode()[0])
        for val in set(data[split_feature]):
            node.children[val]=self.BuildTree(data[data[split_feature]==val])
        return node
 
    def Train(self):
        self.root = self.BuildTree(self.train)

    def Predict(self,node,sample):
        if node.split_feature == None or not node.children:
            return node.cate
        return self.Predict(node.children[sample[node.split_feature]],sample)



In [120]:
tree = ID3(train_data,labels,0.01)
tree.Train()
for idx, sample in train_data.iterrows():
    print(sample['类别']==tree.Predict(tree.root,sample))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [122]:
def dfs(node,tmp):
    if node!= None:
        if node.split_feature:
            tmp = tmp + " "+ node.split_feature
        if not node.split_feature:
            print((tmp+" --> "+node.cate))
    else:
        return
    for k in node.children:
        dfs(node.children[k],tmp+":"+k)
dfs(tree.root,"")

 有自己的房子:否 有工作:否 --> 否
 有自己的房子:否 有工作:是 --> 是
 有自己的房子:是 --> 是
